In [1]:
import pandas as pd 

In [2]:
cols = ('date', 'rate', 'review_text', 'object_name', 'object_address', 'overall_rating')
all_data = pd.read_csv('/Users/camille/repo/Hetic/projet_gouv/scraping/Data/scrape_paris.csv', usecols=cols)
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1027 entries, 0 to 1026
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   date            1027 non-null   object
 1   rate            1027 non-null   int64 
 2   review_text     608 non-null    object
 3   object_name     1027 non-null   object
 4   object_address  1027 non-null   object
 5   overall_rating  1027 non-null   object
dtypes: int64(1), object(5)
memory usage: 48.3+ KB


In [3]:
all_data.dropna(subset=['review_text'], inplace=True)

In [4]:
text = all_data['review_text']


In [5]:
import re
import spacy
import pandas as pd

# Assurez-vous d'exécuter cette ligne pour télécharger le modèle spaCy français
# !python -m spacy download fr_core_news_sm

text = all_data['review_text'].astype(str)

fr = spacy.load("fr_core_news_sm")
stop_words = spacy.lang.fr.stop_words.STOP_WORDS

def clean_text(text):
    # Convertir en minuscules
    text = text.str.lower()
    
    # Enlever les apostrophes
    text = text.str.replace(r'\'', ' ')
    # Enlever les points
    text = text.str.replace(r'\.', ' ')
    # Enlever les virgules
    text = text.str.replace(r',', ' ')

    # Enlever les ponctuations et les caractères spéciaux
    text = text.str.replace(r'[^\w\s]', ' ')
    
    # Enlever les stopwords
    text = text.apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
    
    # Enlever les chiffres et les nombres
    text = text.str.replace(r'\d+', '')
    
    # Enlever les mots de 2 lettres
    text = text.apply(lambda x: ' '.join([word for word in x.split() if len(word) > 2]))

    return text

# Appeler la fonction clean_text() sur la variable text
cleaned_text = clean_text(text)


In [7]:
clean_text = " ".join(cleaned_text) 

In [8]:
import spacy
nlp = spacy.load("fr_core_news_sm")
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer(language='french')
def lemmatize_text(text):
    doc = fr(text)
    lemmatized_sentences = []
    for sent in doc.sents:
        lemmatized_tokens = [token.lemma_ for token in sent]
        lemmatized_sentence = " ".join(lemmatized_tokens)
        lemmatized_sentences.append(lemmatized_sentence)
    return " ".join(lemmatized_sentences)

def  stem_text(text):
    doc = fr(text)
    stem_sentences = []
    for sent in doc.sents:
        stem_tokens = [stemmer.stem(token.text) for token in sent]
        stem_sentence = " ".join(stem_tokens)
        stem_sentences.append(stem_sentence)
    return " ".join(stem_sentences)
stemmed_text = stem_text(clean_text)
clean_stemmed_text = clean_text(stemmed_text)
TypeError: 'str' object is not callable


In [9]:
stemmed_text = stem_text(clean_text)
clean_stemmed_text = clean_text(stemmed_text)
TypeError: 'str' object is not callable


In [14]:
stemmed_text

'agenc totalment désorganis fonction chaotiqu . chang conseil préven matins.vous retrouv conseiller savent vous . l\' approxim l\' incompétent étag malheur pert temp chercheur d\' emploi compt pôl emploi trouv travail bien accueil . bon organis orient l\' on arriv l\' on convoc rdv démarch . meilleur servic client rapid emploi serviabl gentil . bien loin clich efficac compétent hyp symp . merc ! ! équip maximum aid . standard téléphon bout 15minut attent m\' a raccroch nez gentil vrai . espac agréabl professionnel . toutl person cordially . compt utilis moyen informat agenc apport chais n\' est prévu s\' assoir ( sauf l\' uniqu emplac réserv handicap ) armez patienc ordin bêt cours ( san deb réseau simpl navig web consult email ) n\' oubl form utilis solut informat propos non n\' est window traditionnel activ directory outil bureaut traditionnel . derni détail récept réseau l\' agenc aucun wi - fi invit portail captif pali problem demandeur d\' emploi . met étoil personnel ( qui maîtri

In [13]:
clean_stemmed_text = clean_text(stemmed_text)

TypeError: 'str' object is not callable

In [ ]:
cleaned_sentences = [clean_text(sentence) for sentence in streemed_text]
joined_text = " ".join(cleaned_sentences)


In [ ]:
streemed_text

In [ ]:
streemed_text_remove = clean_text(streemed_text)


In [ ]:
from collections import Counter
split_it = streemed_text.split()
word_counts_lem = Counter(split_it)
word_counts_lem

In [ ]:
cleaned_text = cleaned_text(example)

In [ ]:
example = stem_text(clean_text)


In [ ]:
example

In [ ]:

fr = spacy.load("fr_core_news_sm")

def lemmatize_text(text):
    doc = fr(text)
    lemmatized_sentences = []
    for sent in doc.sents:
        lemmatized_tokens = [token.lemma_ for token in sent]
        lemmatized_sentence = " ".join(lemmatized_tokens)
        lemmatized_sentences.append(lemmatized_sentence)
    return " ".join(lemmatized_sentences)

# Appeler la fonction lemmatize_text() sur le texte
lemmatized_text = lemmatize_text(clean_text)

In [ ]:
lemmatized_text

In [ ]:
lemmatized_text
from collections import Counter
split_it = lemmatized_text.split()
word_counts_lem = Counter(split_it)
word_counts_lem

In [ ]:
lemmatized_text

In [ ]:
from PIL import Image
#!pip install wordcloud
from wordcloud import WordCloud
import matplotlib.pyplot as plt
% matplotlib inline

In [ ]:
# Convert the list of words to a string
wordcloud_text = ' '.join(word_counts_lem)

# Create and generate a word cloud image
wordcloud = WordCloud().generate_from_frequencies(word_counts_lem)

# Display the generated image
plt.imshow(wordcloud, interpolation='bilinear')


# Display the generated image:

plt.show()

Prendre un n gram 

In [ ]:
lemmatized_text_modified = ' '.join([word for word in lemmatized_text.split() if len(word) > 3])


In [ ]:
lemmatized_text_modified

In [ ]:
from collections import Counter
split_it = lemmatized_text_modified.split()
word_counts_lem = Counter(split_it)
word_counts_lem

In [ ]:
# Convert the list of words to a string
wordcloud_text = ' '.join(word_counts_lem)

# Create and generate a word cloud image
wordcloud = WordCloud().generate_from_frequencies(word_counts_lem)

# Display the generated image
plt.imshow(wordcloud, interpolation='bilinear')


# Display the generated image:

plt.show()

In [ ]:
import pandas as pd

# Exemple de DataFrame avec une colonne 'text'
data = pd.DataFrame({'text': ['service client, aidez-moi s\'il vous plaît', 'j\'ai besoin d\'aide', 'merci pour votre soutien']})

# Appliquer la transformation sur chaque valeur de la colonne 'text'
data['text'] = data['text'].apply(lambda x: ' '.join([word for word in x.split() if len(word) > 3]))

data


In [ ]:
import re
import spacy
import pandas as pd

# Assurez-vous d'exécuter cette ligne pour télécharger le modèle spaCy français
# !python -m spacy download fr_core_news_sm

text = all_data['review_text'].astype(str)

fr = spacy.load("fr_core_news_sm")
stop_words = spacy.lang.fr.stop_words.STOP_WORDS

def clean_text(text):
    # Convertir en minuscules
    text = text.str.lower()
    
    # Enlever les apostrophes
    text = text.str.replace(r'\'', ' ')
    # Enlever les points
    text = text.str.replace(r'\.', ' ')
    # Enlever les virgules
    text = text.str.replace(r',', ' ')

    # Enlever les ponctuations et les caractères spéciaux
    text = text.str.replace(r'[^\w\s]', ' ')
    
    # Enlever les stopwords
    text = text.apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
    
    # Enlever les chiffres et les nombres
    text = text.str.replace(r'\d+', '')
    
    # Enlever les mots de 2 lettres
    text = text.apply(lambda x: ' '.join([word for word in x.split() if len(word) > 2]))

    return text

# Appeler la fonction clean_text() sur la variable text
cleaned_text = clean_text(text)


In [ ]:
cleaned_text = cleaned_text(exemple)

In [ ]:
cleaned_text

In [ ]:
from collections import Counter

# Utiliser la méthode apply() pour appliquer split() à chaque élément de la série
split_it = cleaned_text.apply(lambda x: x.split())

# Compter les occurrences des mots
word_counts = Counter([word for sublist in split_it for word in sublist])
word_counts


In [15]:
import re
import spacy
import pandas as pd

# Assurez-vous d'exécuter cette ligne pour télécharger le modèle spaCy français
# !python -m spacy download fr_core_news_sm

text = all_data['review_text'].astype(str)

fr = spacy.load("fr_core_news_sm")
stop_words = spacy.lang.fr.stop_words.STOP_WORDS

def clean_text(text):
    # Convertir en minuscules
    text = text.str.lower()
    
    # Enlever les apostrophes
    text = text.str.replace(r'\'', ' ')
    # Enlever les points
    text = text.str.replace(r'\.', ' ')
    # Enlever les virgules
    text = text.str.replace(r',', ' ')

    # Enlever les ponctuations et les caractères spéciaux
    text = text.str.replace(r'[^\w\s]', ' ')
    
    # Enlever les stopwords
    text = text.apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
    
    # Enlever les chiffres et les nombres
    text = text.str.replace(r'\d+', '')
    
    # Enlever les mots de 2 lettres
    text = text.apply(lambda x: ' '.join([word for word in x.split() if len(word) > 2]))

    return text

# Appeler la fonction clean_text() sur la variable text
cleaned_text = clean_text(text)
cleaned_text_str = " ".join(cleaned_text)

import spacy
nlp = spacy.load("fr_core_news_sm")
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer(language='french')

def lemmatize_text(text):
    doc = fr(text)
    lemmatized_sentences = []
    for sent in doc.sents:
        lemmatized_tokens = [token.lemma_ for token in sent]
        lemmatized_sentence = " ".join(lemmatized_tokens)
        lemmatized_sentences.append(lemmatized_sentence)
    return " ".join(lemmatized_sentences)

def  stem_text(text):
    doc = fr(text)
    stem_sentences = []
    for sent in doc.sents:
        stem_tokens = [stemmer.stem(token.text) for token in sent]
        stem_sentence = " ".join(stem_tokens)
        stem_sentences.append(stem_sentence)
    return " ".join(stem_sentences)

stemmed_text = stem_text(cleaned_text_str)
clean_stemmed_text = clean_text(stemmed_text)


AttributeError: 'str' object has no attribute 'str'

In [16]:
import re
import spacy
import pandas as pd

# Assurez-vous d'exécuter cette ligne pour télécharger le modèle spaCy français
# !python -m spacy download fr_core_news_sm

text = all_data['review_text'].astype(str)

fr = spacy.load("fr_core_news_sm")
stop_words = spacy.lang.fr.stop_words.STOP_WORDS

def clean_text(text):
    # Convertir en minuscules
    text = text.lower()
    
    # Enlever les apostrophes
    text = re.sub(r'\'', ' ', text)
    # Enlever les points
    text = re.sub(r'\.', ' ', text)
    # Enlever les virgules
    text = re.sub(r',', ' ', text)

    # Enlever les ponctuations et les caractères spéciaux
    text = re.sub(r'[^\w\s]', ' ', text)
    
    # Enlever les stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])
    
    # Enlever les chiffres et les nombres
    text = re.sub(r'\d+', '', text)
    
    # Enlever les mots de 2 lettres
    text = ' '.join([word for word in text.split() if len(word) > 2])

    return text

# Appeler la fonction clean_text() sur la variable stemmed_text
clean_stemmed_text = clean_text(stemmed_text)


In [17]:
clean_stemmed_text

'agenc totalment désorganis fonction chaotiqu chang conseil préven matins retrouv conseiller savent approxim incompétent étag malheur pert temp chercheur emploi compt pôl emploi trouv travail bien accueil bon organis orient arriv convoc rdv démarch meilleur servic client rapid emploi serviabl gentil bien loin clich efficac compétent hyp symp merc équip maximum aid standard téléphon bout minut attent raccroch nez gentil vrai espac agréabl professionnel toutl person cordially compt utilis moyen informat agenc apport chais prévu assoir uniqu emplac réserv handicap armez patienc ordin bêt cours san deb réseau simpl navig web consult email oubl form utilis solut informat propos non window traditionnel activ directory outil bureaut traditionnel derni détail récept réseau agenc aucun invit portail captif pali problem demandeur emploi met étoil personnel maîtris patient aid utilis utilis chos condit permanent bon accueil bon service conseiller été écout répondu question don précieux conseil be

In [18]:
#counter
from collections import Counter
clean_stemmed_text_counter = Counter(clean_stemmed_text.split())
clean_stemmed_text_counter

Counter({'emploi': 224,
         'accueil': 129,
         'conseil': 128,
         'pôl': 128,
         'conseiller': 96,
         'agenc': 93,
         'fair': 87,
         'demand': 86,
         'travail': 83,
         'bon': 73,
         'bien': 67,
         'servic': 67,
         'person': 67,
         'aucun': 64,
         'été': 63,
         'mois': 61,
         'temp': 57,
         'aid': 56,
         'téléphon': 56,
         'attent': 53,
         'personnel': 51,
         'rien': 51,
         'mail': 50,
         'incompétent': 49,
         'format': 49,
         'dossi': 45,
         'rendez': 43,
         'trouv': 40,
         'jam': 40,
         'trop': 39,
         'fois': 39,
         'expliqu': 38,
         'chang': 37,
         'rdv': 37,
         'faut': 37,
         'répons': 37,
         'désagré': 37,
         'vrai': 35,
         'écout': 35,
         'travaill': 35,
         'répond': 34,
         'appel': 34,
         'pass': 34,
         'inform': 34,
         '

In [ ]:
all_data